In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import TensorBoard, EarlyStopping, ModelCheckpoint
from tensorflow.keras.applications import NASNetMobile, nasnet
from tensorflow.keras.utils import Sequence
import pickle
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Flatten, Dropout, Input, GlobalAveragePooling2D, GlobalMaxPooling2D, Flatten, Concatenate
import orjson
from threading import Thread, Lock
from queue import Queue

import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.


In [2]:
with open('../dataset_test_files.pkl', 'rb') as fp:
    df_test = pickle.load(fp)
df_test.head()

,filename,kind,defect,size,class_name,class
3,D:\desenvolvimento\datasets\TRAIN1200 -1200_CO...,R,S,G,RS,3
4,D:\desenvolvimento\datasets\TRAIN1200 -1200_CO...,R,S,G,RS,3
11,D:\desenvolvimento\datasets\TRAIN1200 -1200_CO...,R,S,G,RS,3
15,D:\desenvolvimento\datasets\TRAIN1200 -1200_CO...,R,S,G,RS,3
20,D:\desenvolvimento\datasets\TRAIN1200 -1200_CO...,R,S,G,RS,3


In [3]:
with open('../dataset_train_files.pkl', 'rb') as fp:
    df_train = pickle.load(fp)
df_train.head()

,filename,kind,defect,size,class_name,class
0,D:\desenvolvimento\datasets\TRAIN1200 -1200_CO...,R,S,G,RS,3
1,D:\desenvolvimento\datasets\TRAIN1200 -1200_CO...,R,S,G,RS,3
2,D:\desenvolvimento\datasets\TRAIN1200 -1200_CO...,R,S,M,RS,3
5,D:\desenvolvimento\datasets\TRAIN1200 -1200_CO...,R,S,M,RS,3
6,D:\desenvolvimento\datasets\TRAIN1200 -1200_CO...,R,S,G,RS,3


In [4]:
class ImageGeneratorSplited(Sequence):
    def __init__(self, df, y_col='class_name', batch_size=64, split=0.5, queue_size=100, **kwargs):
        self.gen_da = ImageDataGenerator(**kwargs)
        self.gen_raw = ImageDataGenerator()
        self.length = int(len(df.index) / (batch_size * (1 - split)))
        self.batch_size = batch_size
        self.batch_da = int(batch_size * split)
        self.flow_da = self.gen_da.flow_from_dataframe(df, y_col=y_col, batch_size=self.batch_da)
        self.batch_raw = self.batch_size - self.batch_da
        self.flow_raw = self.gen_raw.flow_from_dataframe(df, y_col=y_col, batch_size=self.batch_raw)
        temp = next(self.flow_raw)
        self.shape = (self.length, *temp[0].shape[1:])
        self._next_da = Queue(queue_size)
        self._next_raw = Queue(queue_size)
        self._next_da_thread = self.prepare_da()
        self._next_raw_thread = self.prepare_raw()
        
    def __len__(self):
        return self.length
        
    def get_next_da(self):
        return self._next_da.get()
    
    def get_next_raw(self):
        return self._next_raw.get()
    
    def _prepare_next_da(self):
        while True:
            self._next_da.put(next(self.flow_da))
    
    def _prepare_next_raw(self):
        while True:
            self._next_raw.put(next(self.flow_raw))
            
    def prepare_da(self):
        t = Thread(target=self._prepare_next_da)
        t.start()
        return t
    
    def prepare_raw(self):
        t = Thread(target=self._prepare_next_raw)
        t.start()
        return t
    
    
    def __iter__(self):
        for _ in range(len(self)):
            yield self[None]

    def __getitem__(self, item):
        x_raw, y_raw = self.get_next_raw()
        x_da, y_da = self.get_next_da()
        return nasnet.preprocess_input(np.vstack((x_raw, x_da))), np.vstack((y_raw, y_da))
    

In [5]:
SPLIT = 0.7
gen_train = ImageGeneratorSplited(df_train,
    rotation_range=90,
    width_shift_range=0.2,
    height_shift_range=0.2,
    # zoom_range=[1, 3],
    horizontal_flip=True,
    vertical_flip=True,
    split=SPLIT,
    batch_size=64
)
print(gen_train)
gen_test = ImageGeneratorSplited(df_test, split=SPLIT, batch_size=128)
gen_test

Found 840 validated image filenames belonging to 4 classes.
Found 840 validated image filenames belonging to 4 classes.
Found 360 validated image filenames belonging to 4 classes.
Found 360 validated image filenames belonging to 4 classes.


In [6]:
SHAPE = (256, 256, 3)
input_tensor = Input(shape=SHAPE)
base_model = NASNetMobile(SHAPE, False, None, input_tensor=input_tensor)
base_model.summary()

Model: "NASNet"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
stem_conv1 (Conv2D)             (None, 127, 127, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
stem_bn1 (BatchNormalization)   (None, 127, 127, 32) 128         stem_conv1[0][0]                 
__________________________________________________________________________________________________
activation (Activation)         (None, 127, 127, 32) 0           stem_bn1[0][0]                   
_____________________________________________________________________________________________

In [7]:
x = base_model.output
out1 = GlobalMaxPooling2D()(x)
out2 = GlobalAveragePooling2D()(x)
out3 = Flatten()(x)
out = Concatenate(axis=-1)([out1, out2, out3])
out = Dropout(0.5)(out)
predictions = Dense(4, activation="sigmoid", name="3_")(out)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
stem_conv1 (Conv2D)             (None, 127, 127, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
stem_bn1 (BatchNormalization)   (None, 127, 127, 32) 128         stem_conv1[0][0]                 
__________________________________________________________________________________________________
activation (Activation)         (None, 127, 127, 32) 0           stem_bn1[0][0]                   
______________________________________________________________________________________________

In [8]:
model.compile(optimizer='adam', # Rotina de otimização, que informa a ml como ajustar o valor dos parâmetros para minimizar o erro.
              loss='categorical_crossentropy', # Função de erro que diz o quão erradas estão nossas predições
              metrics=['accuracy']) # Lista de métricas para avaliar o nosso modelo

In [9]:
with open('model_da_nasnetmobile.json', 'w') as fp:
    fp.write(model.to_json())

In [10]:
history = model.fit(
    gen_train, 
    epochs=100, 
    verbose=2, 
    validation_data=gen_test, 
    callbacks=[
    ModelCheckpoint(
        'model_da_nasnetmobile.{epoch:03d}-{val_accuracy:.4f}.h5',
        monitor='val_accuracy',
        save_best_only=True,
        save_weights_only=True,
    ),
    TensorBoard(log_dir='logs_da_nasnetmobile'),
])

Train for 43 steps, validate for 9 steps
Epoch 1/100
43/43 - 55s - loss: 1.6688 - accuracy: 0.2537 - val_loss: 1.3863 - val_accuracy: 0.2468
Epoch 2/100
43/43 - 24s - loss: 1.3863 - accuracy: 0.2466 - val_loss: 1.3863 - val_accuracy: 0.2521
Epoch 3/100
43/43 - 23s - loss: 1.3863 - accuracy: 0.2515 - val_loss: 1.3863 - val_accuracy: 0.2517
Epoch 4/100
43/43 - 24s - loss: 1.3863 - accuracy: 0.2496 - val_loss: 1.3863 - val_accuracy: 0.2495
Epoch 5/100
43/43 - 24s - loss: 1.3863 - accuracy: 0.2537 - val_loss: 1.3863 - val_accuracy: 0.2405
Epoch 6/100
43/43 - 24s - loss: 1.3863 - accuracy: 0.2454 - val_loss: 1.3863 - val_accuracy: 0.2595
Epoch 7/100
43/43 - 24s - loss: 1.3863 - accuracy: 0.2481 - val_loss: 1.3863 - val_accuracy: 0.2489
Epoch 8/100
43/43 - 24s - loss: 1.3863 - accuracy: 0.2534 - val_loss: 1.3863 - val_accuracy: 0.2555
Epoch 9/100
43/43 - 24s - loss: 1.3863 - accuracy: 0.2519 - val_loss: 1.3863 - val_accuracy: 0.2447
Epoch 10/100
43/43 - 24s - loss: 1.3863 - accuracy: 0.2496 

KeyError: 'val_accuracy'

In [ ]:
with open('train_da_history.json', 'wb') as fp:
    fp.write(orjson.dumps({k: np.array(v).tolist() for k, v in history.history.items()}))

In [ ]:
history2 = model.fit(
    gen_train, 
    epochs=100, 
    verbose=2, 
    validation_data=gen_test, 
    callbacks=[
    ModelCheckpoint(
        'model_da_nasnetmobile.{epoch:03d}-{val_accuracy:.4f}.h5',
        monitor='val_accuracy',
        save_best_only=True,
        save_weights_only=True,
    ),
    TensorBoard(log_dir='logs_da_nasnetmobile'),
])

In [ ]:
with open('train_da_history2.json', 'wb') as fp:
    fp.write(orjson.dumps({k: np.array(v).tolist() for k, v in history2.history.items()}))